In [1]:
# Install required packages
!pip install ultralytics xmltodict wandb -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.2/922.2 kB 16.3 MB/s eta 0:00:00


In [2]:
import os
import shutil
import random

dataset_path = "/kaggle/input/sccos-dataset/"
working_dir = "/kaggle/working/yolo_dataset"
train_images_dir = os.path.join(working_dir, "train/images")
train_labels_dir = os.path.join(working_dir, "train/labels")
val_images_dir = os.path.join(working_dir, "val/images")
val_labels_dir = os.path.join(working_dir, "val/labels")
test_images_dir = os.path.join(working_dir, "test/images")
test_labels_dir = os.path.join(working_dir, "test/labels")

# Clean and recreate directories
if os.path.exists(working_dir):
    shutil.rmtree(working_dir)
for dir_path in [train_images_dir, train_labels_dir, val_images_dir, val_labels_dir, test_images_dir, test_labels_dir]:
    os.makedirs(dir_path, exist_ok=True)

# Source paths
src_img_dir = os.path.join(dataset_path, "train/images")
src_ann_dir = os.path.join(dataset_path, "train/annotations")

# Get all image files
image_files = [f for f in os.listdir(src_img_dir) if f.endswith('.png')]
random.seed(42)  # For reproducibility
random.shuffle(image_files)

# Split ratios: 70% train, 20% val, 10% test
total_size = len(image_files)
train_size = int(0.8 * total_size)  # ~2597 images
val_size = int(0.1 * total_size)    # ~742 images
test_size = total_size - train_size - val_size  # ~372 images

train_files = image_files[:train_size]
val_files = image_files[train_size:train_size + val_size]
test_files = image_files[train_size + val_size:]

# Function to copy files
def copy_files(file_list, src_img_dir, src_ann_dir, dst_img_dir, dst_ann_dir):
    for img_file in file_list:
        shutil.copy(os.path.join(src_img_dir, img_file), os.path.join(dst_img_dir, img_file))
        ann_file = img_file.replace('.png', '.xml')
        if os.path.exists(os.path.join(src_ann_dir, ann_file)):
            shutil.copy(os.path.join(src_ann_dir, ann_file), os.path.join(dst_ann_dir, ann_file))

# Copy files to respective directories
copy_files(train_files, src_img_dir, src_ann_dir, train_images_dir, train_labels_dir)
copy_files(val_files, src_img_dir, src_ann_dir, val_images_dir, val_labels_dir)
copy_files(test_files, src_img_dir, src_ann_dir, test_images_dir, test_labels_dir)

print(f"Dataset split: Train={len(train_files)}, Val={len(val_files)}, Test={len(test_files)}")

Dataset split: Train=2968, Val=371, Test=372


In [3]:
import xmltodict
import tqdm
import math

def convert_robndbox_to_corners(cx, cy, w, h, angle):
    """Convert center-based robndbox to 4 corner points."""
    angle = float(angle)
    cx, cy, w, h = float(cx), float(cy), float(w), float(h)
    cos_a = math.cos(angle)
    sin_a = math.sin(angle)

    dx, dy = w / 2, h / 2
    corners = [(-dx, -dy), (dx, -dy), (dx, dy), (-dx, dy)]
    rotated_corners = [
        (cx + x * cos_a - y * sin_a, cy + x * sin_a + y * cos_a)
        for x, y in corners
    ]
    return rotated_corners

def convert_xml_to_yolo_obb_corners(xml_path, img_path, output_img_dir, output_label_dir):
    img_filename = os.path.splitext(os.path.basename(img_path))[0] + ".png"
    # No need to copy image here since it's already copied in Cell 2

    with open(xml_path, 'r') as f:
        xml_data = xmltodict.parse(f.read())
    objects = xml_data['annotation'].get('object', [])
    if not isinstance(objects, list):
        objects = [objects] if objects else []

    img_size = 1024  # SCCOS dataset image size
    txt_lines = []
    for obj in objects:
        if obj and 'robndbox' in obj:
            robndbox = obj['robndbox']
            try:
                cx = float(robndbox['cx'])
                cy = float(robndbox['cy'])
                w = float(robndbox['w'])
                h = float(robndbox['h'])
                angle = float(robndbox['angle'])
                (x1, y1), (x2, y2), (x3, y3), (x4, y4) = convert_robndbox_to_corners(cx, cy, w, h, angle)
                
                # Normalize coordinates
                x1, y1 = x1 / img_size, y1 / img_size
                x2, y2 = x2 / img_size, y2 / img_size
                x3, y3 = x3 / img_size, y3 / img_size
                x4, y4 = x4 / img_size, y4 / img_size
                
                class_id = 0  # "ship" as class 0
                txt_lines.append(f"{class_id} {x1:.6f} {y1:.6f} {x2:.6f} {y2:.6f} {x3:.6f} {y3:.6f} {x4:.6f} {y4:.6f}")
            except (KeyError, ValueError) as e:
                print(f"Invalid robndbox data in {xml_path}: {e}")
                continue

    txt_filename = os.path.splitext(os.path.basename(xml_path))[0] + ".txt"
    txt_path = os.path.join(output_label_dir, txt_filename)
    if txt_lines:
        with open(txt_path, 'w') as f:
            f.write("\n".join(txt_lines))
    else:
        print(f"No valid objects in {xml_path}, skipping label file.")

# Process train, val, and test sets
for split, img_dir, label_dir in [
    ("train", train_images_dir, train_labels_dir),
    ("val", val_images_dir, val_labels_dir),
    ("test", test_images_dir, test_labels_dir)
]:
    print(f"Converting {split} set...")
    for xml_file in tqdm.tqdm(os.listdir(label_dir)):
        if xml_file.endswith(".xml"):
            img_file = os.path.join(img_dir, xml_file.replace(".xml", ".png"))
            if os.path.exists(img_file):
                convert_xml_to_yolo_obb_corners(
                    os.path.join(label_dir, xml_file),
                    img_file,
                    img_dir,
                    label_dir
                )
            # Remove XML file after conversion
            os.remove(os.path.join(label_dir, xml_file))

print("Dataset conversion complete.")

Converting train set...


100%|██████████| 2968/2968 [00:01<00:00, 2128.40it/s]


Converting val set...


100%|██████████| 371/371 [00:00<00:00, 2024.64it/s]


Converting test set...


100%|██████████| 372/372 [00:00<00:00, 2103.92it/s]

Dataset conversion complete.


In [4]:
for split, img_dir, label_dir in [
    ("train", train_images_dir, train_labels_dir),
    ("val", val_images_dir, val_labels_dir),
    ("test", test_images_dir, test_labels_dir)
]:
    images = len(os.listdir(img_dir))
    labels = len(os.listdir(label_dir))
    print(f"{split.capitalize()} images: {images}")
    print(f"{split.capitalize()} labels: {labels}")

# Check sample label
sample_label = os.path.join(train_labels_dir, "0008.txt")
if os.path.exists(sample_label):
    with open(sample_label, 'r') as f:
        content = f.read().strip()
        print(f"Sample label (0008.txt):")
        print(content)
        columns = content.splitlines()[0].split()
        print(f"Number of columns: {len(columns)}")
else:
    print(f"Sample label {sample_label} not found!")

Train images: 2968
Train labels: 2968
Val images: 371
Val labels: 371
Test images: 372
Test labels: 372
Sample label (0008.txt):
0 0.852496 0.235036 0.918159 0.249720 0.809446 0.735872 0.743783 0.721188
0 0.080527 0.900865 0.098952 0.914299 0.053253 0.976977 0.034828 0.963543
0 0.131338 0.832076 0.145384 0.842317 0.109178 0.891975 0.095132 0.881734
0 0.274821 0.567486 0.308875 0.589797 0.164564 0.810063 0.130509 0.787751
Number of columns: 9


In [5]:
import yaml

dataset_config = {
    "path": working_dir,
    "train": "train/images",
    "val": "val/images",
    "test": "test/images",
    "nc": 1,
    "names": ["ship"]
}

yaml_path = os.path.join(working_dir, "dataset.yaml")
with open(yaml_path, "w") as f:
    yaml.dump(dataset_config, f)

!cat {yaml_path}

names:
- ship
nc: 1
path: /kaggle/working/yolo_dataset
test: test/images
train: train/images
val: val/images


In [6]:
import wandb
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("wandb_api_key")
wandb.login(key=wandb_api_key)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: tanish-jdh2020. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
from ultralytics import YOLO
import os

# Clear cache
for split in ["train", "val", "test"]:
    cache_file = os.path.join(working_dir, f"{split}/labels.cache")
    if os.path.exists(cache_file):
        os.remove(cache_file)
        print(f"Cleared {split} label cache.")

# Load and train
model = YOLO("yolov8n-obb.pt")  # Pretrained OBB model

# Train with all logging integrations disabled
model.train(
    data=yaml_path,
    task="obb",
    epochs=50,
    imgsz=1024,
    batch=8,
    device=0,
    verbose=True,
    project="/kaggle/working/runs",
    name="yolov8-obb-train",
    patience=10,
    save_period=5,
    workers=4,
    cache=False,
    seed=42,
             # Disable W&B (remove if you want to use it)
)

print("Training completed.")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.26M/6.26M [00:00<00:00, 83.5MB/s]


Ultralytics 8.3.82 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=obb, mode=train, model=yolov8n-obb.pt, data=/kaggle/working/yolo_dataset/dataset.yaml, epochs=50, time=None, patience=10, batch=8, imgsz=1024, save=True, save_period=5, cache=False, device=0, workers=4, project=/kaggle/working/runs, name=yolov8-obb-train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=

100%|██████████| 755k/755k [00:00<00:00, 18.2MB/s]


Overriding model.yaml nc=15 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...


50 epochs completed in 1.382 hours.
Optimizer stripped from /kaggle/working/runs/yolov8-obb-train/weights/last.pt, 6.6MB
Optimizer stripped from /kaggle/working/runs/yolov8-obb-train/weights/best.pt, 6.6MB

Validating /kaggle/working/runs/yolov8-obb-train/weights/best.pt...
Ultralytics 8.3.82 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8n-obb summary (fused): 81 layers, 3,077,414 parameters, 0 gradients, 8.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 47/47 [00:07<00:00,  6.21it/s]


                   all        371       1815       0.87      0.861      0.926      0.762


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.6ms preprocess, 5.8ms inference, 0.0ms loss, 4.3ms postprocess per image
Results saved to /kaggle/working/runs/yolov8-obb-train
Training completed.


In [8]:
metrics = model.val(task="obb", split="val")
print(f"Validation mAP@0.5: {metrics.box.map50:.4f}")
print(f"Validation mAP@0.5:0.95: {metrics.box.map:.4f}")
print("Validation metrics:", metrics)

Ultralytics 8.3.82 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8n-obb summary (fused): 81 layers, 3,077,414 parameters, 0 gradients, 8.3 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 371 images, 0 backgrounds, 0 corrupt: 100%|██████████| 371/371 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 47/47 [00:08<00:00,  5.31it/s]


                   all        371       1815       0.87      0.861      0.926      0.763


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 2.5ms preprocess, 6.0ms inference, 0.0ms loss, 3.4ms postprocess per image
Results saved to /kaggle/working/runs/yolov8-obb-train2
Validation mAP@0.5: 0.9258
Validation mAP@0.5:0.95: 0.7631
Validation metrics: ultralytics.utils.metrics.OBBMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79f19bada5c0>
curves: []
curves_results: []
fitness: 0.7793887641258656
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.76313])
names: {0: 'ship'}
plot: True
results_dict: {'metrics/precision(B)': 0.8697928569569083, 'metrics/recall(B)': 0.8612322280730677, 'metrics/mAP50(B)': 0.9257593661760106, 'metrics/mAP50-95(B)': 0.7631253638980716, 'fitness': 0.7793887641258656}
save_dir: PosixPath('/kaggle/working/runs/yolov8-obb-train2')
speed: {'preprocess': 2.493921121297391, 'inference': 6.01499431806558, 'los

In [9]:
test_metrics = model.val(task="obb", split="test")
print(f"Test mAP@0.5: {test_metrics.box.map50:.4f}")
print(f"Test mAP@0.5:0.95: {test_metrics.box.map:.4f}")
print("Test metrics:", test_metrics)

Ultralytics 8.3.82 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/working/yolo_dataset/test/labels... 372 images, 0 backgrounds, 0 corrupt: 100%|██████████| 372/372 [00:02<00:00, 132.71it/s]

val: New cache created: /kaggle/working/yolo_dataset/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 47/47 [00:08<00:00,  5.31it/s]


                   all        372       1573      0.858      0.866      0.933      0.781


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 1.5ms preprocess, 6.0ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to /kaggle/working/runs/yolov8-obb-train3
Test mAP@0.5: 0.9327
Test mAP@0.5:0.95: 0.7811
Test metrics: ultralytics.utils.metrics.OBBMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79f19bb27490>
curves: []
curves_results: []
fitness: 0.7962860840829062
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.78113])
names: {0: 'ship'}
plot: True
results_dict: {'metrics/precision(B)': 0.8578301104063907, 'metrics/recall(B)': 0.866497139224412, 'metrics/mAP50(B)': 0.9326935413572648, 'metrics/mAP50-95(B)': 0.7811296999413109, 'fitness': 0.7962860840829062}
save_dir: PosixPath('/kaggle/working/runs/yolov8-obb-train3')
speed: {'preprocess': 1.475803548395737, 'inference': 5.959981553746485, 'loss': 0.002429803759

In [10]:
model.export(format="torchscript")
print("Model exported to TorchScript format.")

Ultralytics 8.3.82 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.00GHz)

PyTorch: starting from '/kaggle/working/runs/yolov8-obb-train/weights/best.pt' with input shape (1, 3, 1024, 1024) BCHW and output shape(s) (1, 6, 21504) (6.3 MB)

TorchScript: starting export with torch 2.5.1+cu121...
TorchScript: export success ✅ 2.6s, saved as '/kaggle/working/runs/yolov8-obb-train/weights/best.torchscript' (12.4 MB)

Export complete (4.4s)
Results saved to /kaggle/working/runs/yolov8-obb-train/weights
Predict:         yolo predict task=obb model=/kaggle/working/runs/yolov8-obb-train/weights/best.torchscript imgsz=1024  
Validate:        yolo val task=obb model=/kaggle/working/runs/yolov8-obb-train/weights/best.torchscript imgsz=1024 data=/kaggle/working/yolo_dataset/dataset.yaml  
Visualize:       https://netron.app
Model exported to TorchScript format.
